# Model Training

Prepare dataframe 

In [6]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from nltk import sent_tokenize
from gensim.models import Word2Vec
from sklearn.svm import LinearSVC, SVC
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from utils.text_preprocess import LemmaTokenizer

df = pd.read_csv('data/amazon_reviews.txt', sep='\t')
df['VERIFIED_PURCHASE'] = df['VERIFIED_PURCHASE'].map({'Y': 1, 'N': 0})

word2vec_model_file = 'models/word2vec_200.model'
model = Word2Vec.load(word2vec_model_file)

def average_word_embeddings(sentence, model):
    words = LemmaTokenizer()(sentence)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    
    return np.mean(word_vectors, axis=0)

# Define a function to process each review
def process_review(review_text, model):
    sentences = sent_tokenize(review_text)
    sentence_embeddings = [average_word_embeddings(sentence, model) for sentence in sentences]
    return sentence_embeddings

df['Sentence_Embeddings'] = df['REVIEW_TEXT'].apply(lambda x: process_review(x, model))


E:\PycharmProjects\amazon_fake_reviews_detection\utils\text_preprocess.py:30: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


Create pipelines

In [7]:
from utils.transformers import *
    
classifiers = [
    RandomForestClassifier(random_state=42, n_jobs=-1),
    CalibratedClassifierCV(LinearSVC(random_state=42, dual=False)),
    MultinomialNB(),
]
vectorizers = [
    TfidfVectorizer(tokenizer=LemmaTokenizer(), token_pattern=None),
    CountVectorizer(tokenizer=LemmaTokenizer(), token_pattern=None),
]

pipes = []
pipes_names = []
for clf in classifiers:
    for vectorizer in vectorizers:
        for useEmbeddings in [True, False]:
            for useVP in [True, False]:
                transformers = [
                    ('vectorizer', vectorizer, 'REVIEW_TEXT')
                ]
                if useEmbeddings:
                    transformers.append(('embeddings', SentenceEmbeddingTransformer(), 'Sentence_Embeddings'))
                if useVP:
                    transformers.append(('encoder', OneHotEncoder(), ['VERIFIED_PURCHASE']))
                
                if isinstance(clf, MultinomialNB):
                    # Add DenseTransformer after ColumnTransformer to convert the full sparse matrix to dense
                    pipes.append(Pipeline([
                        ('preprocessor', ColumnTransformer(transformers, n_jobs=-1)),
                        ('to_dense', DenseTransformer()),  # Apply DenseTransformer here
                        ('clf', clf)
                    ]))
                else:
                    # No need for DenseTransformer for other classifiers
                    pipes.append(Pipeline([
                        ('preprocessor', ColumnTransformer(transformers, n_jobs=-1)),
                        ('clf', clf)
                    ]))
                    
                pipes_names.append(f'{clf.__class__.__name__}_'
                                   f'{vectorizer.__class__.__name__}'
                                   f'{"_WE" if useEmbeddings else ""}'
                                   f'{"_VP" if useVP else ""}')

Nested cross-validation

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from nltk import sent_tokenize
import pandas as pd
import joblib
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import cross_val_score, HalvingGridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from utils.text_preprocess import LemmaTokenizer


param_grids = {
    'RandomForestClassifier': {
        'clf__n_estimators': [800, 1000, 1200],
        'clf__criterion': ['entropy', 'log_loss'],
        'clf__max_depth': [None, 20, 40, 60],
        'clf__bootstrap': [False, True],
    },
    'CalibratedClassifierCV': {
        'clf__estimator__C': [0.5, 1, 1.5],
        'clf__estimator__max_iter': [2000, 4000],
        'clf__estimator__loss': ['hinge', 'squared_hinge'],
    },
    'MultinomialNB': {
        'clf__alpha': [0.0001, 0.001, 0.1, 0.5, 1.0],
    }
}

inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results = {}
best_accuracy = 0
best_model = None

for pipe, pipe_name in zip(pipes, pipes_names):
    print(pipe_name)
    clf_name = pipe.steps[-1][1].__class__.__name__
    grid = param_grids[clf_name]
    clf = HalvingGridSearchCV(pipe, grid, cv=inner_cv, factor=2, random_state=42, n_jobs=-1)
    results[pipe_name] = [[], [], [], []]
    
    # Loop instead of cross_val_score to have more control over printing
    for fold, (train_idx, test_idx) in enumerate(outer_cv.split(df, df['LABEL'])):
        print(f"Fold {fold}:")
        
        # Split the data
        X_train, X_test = df.iloc[train_idx], df.iloc[test_idx]
        y_train, y_test = df['LABEL'].iloc[train_idx], df['LABEL'].iloc[test_idx]

        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        results[pipe_name][0].append(accuracy_score(y_test, y_pred))
        results[pipe_name][1].append(f1_score(y_test, y_pred, pos_label='__label1__'))
        results[pipe_name][2].append(roc_auc_score(y_test, clf.predict_proba(X_test)[:,1]))
        results[pipe_name][3].append(clf.best_params_)
        
        # Update best model 
        if results[pipe_name][0][-1] > best_accuracy:
            best_accuracy = results[pipe_name][0][-1]
            best_model = clf.best_estimator_

        print(f"Best params: {clf.best_params_}\n"
              f"Test accuracy: { results[pipe_name][0][-1]}\n"
              f"Test F1: { results[pipe_name][1][-1]}\n"
              f"Test AUC: { results[pipe_name][2][-1]}\n")
    
    print(f'Mean Accuracy: {np.mean(results[pipe_name][0])}\n'
          f'Mean F1: {np.mean(results[pipe_name][1])}\n'
          f'Mean AUC: {np.mean(results[pipe_name][2])}\n')

# save results to csv file
results_str = {key: [str(value) for value in values] for key, values in results.items()}

df = pd.DataFrame()
for key, value in results_str.items():
    temp_df = pd.DataFrame([value], columns=['accuracy', 'f1_score', 'AUC', 'hyperparameters'])
    temp_df.insert(0, 'model', key)
    df = pd.concat([df, temp_df], ignore_index=True)
df.to_csv('results.csv', index=False)

# save best model
joblib.dump(best_model, 'best_model.pkl')

RandomForestClassifier_TfidfVectorizer_WE_VP
Fold 0:
Best params: {'clf__bootstrap': False, 'clf__criterion': 'entropy', 'clf__max_depth': 60, 'clf__n_estimators': 1000}
Test accuracy: 0.7980952380952381
Test F1: 0.7965451055662188
Test AUC: 0.8655809523809525

Fold 1:
Best params: {'clf__bootstrap': True, 'clf__criterion': 'log_loss', 'clf__max_depth': 40, 'clf__n_estimators': 1200}
Test accuracy: 0.7976190476190477
Test F1: 0.7927840078010727
Test AUC: 0.8639555555555556

Fold 2:
Best params: {'clf__bootstrap': False, 'clf__criterion': 'entropy', 'clf__max_depth': 40, 'clf__n_estimators': 1000}
Test accuracy: 0.8076190476190476
Test F1: 0.804642166344294
Test AUC: 0.8682113378684807

Fold 3:
Best params: {'clf__bootstrap': True, 'clf__criterion': 'log_loss', 'clf__max_depth': 60, 'clf__n_estimators': 1200}
Test accuracy: 0.7880952380952381
Test F1: 0.7838756677999028
Test AUC: 0.8615224489795918

Fold 4:
Best params: {'clf__bootstrap': False, 'clf__criterion': 'entropy', 'clf__max_de

C:\Users\Utente\anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best params: {'clf__bootstrap': False, 'clf__criterion': 'log_loss', 'clf__max_depth': None, 'clf__n_estimators': 1200}
Test accuracy: 0.6519047619047619
Test F1: 0.6623556581986143
Test AUC: 0.7151020408163266

Fold 5:
Best params: {'clf__bootstrap': True, 'clf__criterion': 'entropy', 'clf__max_depth': 40, 'clf__n_estimators': 800}
Test accuracy: 0.6628571428571428
Test F1: 0.6761207685269899
Test AUC: 0.739482993197279

Fold 6:
Best params: {'clf__bootstrap': True, 'clf__criterion': 'log_loss', 'clf__max_depth': 40, 'clf__n_estimators': 1200}
Test accuracy: 0.6461904761904762
Test F1: 0.6666666666666667
Test AUC: 0.7089269841269842

Fold 7:
Best params: {'clf__bootstrap': True, 'clf__criterion': 'log_loss', 'clf__max_depth': 60, 'clf__n_estimators': 1200}
Test accuracy: 0.66
Test F1: 0.6721763085399449
Test AUC: 0.7242585034013606

Fold 8:
Best params: {'clf__bootstrap': True, 'clf__criterion': 'log_loss', 'clf__max_depth': 40, 'clf__n_estimators': 1000}
Test accuracy: 0.640476190476